# Филогенетический анализ

Bacillus subtilis

Суммарное число последовательностей 57

Гены для конкатенации: 
gyrA
guaB
folB
secE
rpoB
rpoC
truA
purT
nadE
rsbW
moaC
purE
purK
purL
purN
purD
gatC
thiC
hemE
hemH
addB
argJ
argB
thiS
thiG
thiD
uxuA
purU
proA
pyrF
recG
ftsY
rimM
trmD
codY
flgB
fliE
fliG
fliJ
fliM
fliP
flhA
flhF
dxr
nusA
rbfA
truB
asd
dapA
tdh
mutS

In [1]:
import os
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
def get_cds_feature_with_qualifier_value(seq_record, value):
    """Function to look for CDS feature by annotation value in sequence record.
    """
    # Loop over the features
    for rec in SeqIO.parse(seq_record, "genbank"):
        for feature in rec.features:
            if feature.type == "CDS" and value in feature.qualifiers.get('gene', []):
                return feature.location.extract(rec.seq)
    # Could not find it
    return None

In [55]:
list_of_genes = ['dapA', 'tdh', 'mutS']

In [41]:
list_of_genes = ['gyrA', 'guaB', 'folB', 'secE', 'rpoB', 'rpoC', 'truA', 'purT', 'nadE', 'rsbW', 'moaC', 'purE', 'purK', 'purL', 'purN', 'purD', 'gatC', 'thiC', 'hemE', 'hemH', 'addB', 'argJ', 'argB', 'thiS', 'thiG', 'thiD', 'uxuA', 'purU', 'proA', 'pyrF', 'recG', 'ftsY', 'rimM', 'trmD', 'codY', 'flgB', 'fliE', 'fliG', 'fliJ', 'fliM', 'fliP', 'flhA', 'flhF', 'dxr', 'nusA', 'rbfA', 'truB', 'asd', 'dapA', 'tdh', 'mutS']

In [56]:
# Получаем фасту gyrA_Bacillus.fasta со gyrA генами
path = '/Users/lolitiy/Documents/Education/inst_bioinf_19_20/prok'
for dir_name, d, files in os.walk(path):
    # перебираем все гены из списка и создаем "gene"_Bacillus.fasta файлы
    for gene in list_of_genes:
        records = []
        for file in files:
            if 'gbff' in file:
                path_fasta = path + '/' + file
                cds_feature = str(get_cds_feature_with_qualifier_value(path_fasta, gene))
                record = SeqRecord(Seq(cds_feature.strip("''")),
                               id=file.strip('.gbff'),
                               name=gene,
                               description=gene)
                records.append(record)
        SeqIO.write(records, "{}_Bacillus.fasta".format(gene), "fasta")
            

### Выравниваем последовательности относительно друг друга 

In [53]:
!for i in *.fasta \ do \ mafft ${i}_Bacillus.fasta > ${i}_Bacillus_al.fasta \ done

/bin/bash: -c: line 0: syntax error near unexpected token `>'
/bin/bash: -c: line 0: `for i in *.fasta \ do \ mafft ${i}_Bacillus.fasta > ${i}_Bacillus_al.fasta \ done'


### Делаем конкатенат из 50 fasta файлов

In [1]:
!cat *_al.fasta > full_al.fasta

In [9]:
from Bio import SeqIO
from collections import defaultdict
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

sequence_map = defaultdict(str)

for sequence in SeqIO.parse('full_al.fasta', "fasta"):
    sequence_map[sequence.name] += str(sequence.seq)

In [13]:
records = []
for key in sorted(sequence_map.keys()):
    record = SeqRecord(Seq(sequence_map[key]),
                       id=key,
                      description='concatenate')
    
    records.append(record)
    SeqIO.write(records, "all_concat.fasta", "fasta")

### Строим деревья с помощью http://iqtree.cibiv.univie.ac.at/  
#iqtree -s all_concat.fasta -m TEST -bb 1000 -alrt 1000 

- На основании сравнения 1000 альтернативных деревьев  
- Алгоритм построения Maximum-likelihood  
- Best-fit model: GTR+F+I+G4 chosen according to BIC  

Consensus tree written to all_concat.fasta.contree

### Визуализация филогенетического дерева с помощью https://itol.embl.de/ с bootstrap > 80%

![](RvczxF85-uoHP6virlUg-w.png)